In [20]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alxmamaev/flowers-recognition")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/flowers-recognition


In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [22]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [23]:
train = datagen.flow_from_directory(
    '/kaggle/input/flowers-recognition/flowers',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Load validation images
val = datagen.flow_from_directory(
    '/kaggle/input/flowers-recognition/flowers',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

Found 3457 images belonging to 5 classes.
Found 860 images belonging to 5 classes.


In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [25]:
CNN_model = Sequential()
CNN_model.add(Input(shape=(128, 128, 3)))
CNN_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
CNN_model.add(MaxPooling2D((2, 2)))
CNN_model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
CNN_model.add(MaxPooling2D((2, 2)))
CNN_model.add(Flatten())
CNN_model.add(Dense(512,activation='relu'))
CNN_model.add(Dense(256,activation='relu'))
CNN_model.add(Dense(128,activation='relu'))
CNN_model.add(Dense(5, activation='softmax'))
CNN_model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
CNN_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │    16,777,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,962,853 (64.71 MB)

 Trainable params: 16,962,853 (64.71 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
CNN_model.fit(train, validation_data=val, epochs=5)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
109/109 ━━━━━━━━━━━━━━━━━━━━ 34s 282ms/step - accuracy: 0.3109 - loss: 1.5906 - val_accuracy: 0.4651 - val_loss: 1.2632
Epoch 2/5
109/109 ━━━━━━━━━━━━━━━━━━━━ 29s 266ms/step - accuracy: 0.5096 - loss: 1.1965 - val_accuracy: 0.5163 - val_loss: 1.2033
Epoch 3/5
109/109 ━━━━━━━━━━━━━━━━━━━━ 28s 257ms/step - accuracy: 0.5575 - loss: 1.0981 - val_accuracy: 0.5884 - val_loss: 1.0007
Epoch 4/5
109/109 ━━━━━━━━━━━━━━━━━━━━ 28s 257ms/step - accuracy: 0.6099 - loss: 0.9731 - val_accuracy: 0.5884 - val_loss: 1.0106
Epoch 5/5
109/109 ━━━━━━━━━━━━━━━━━━━━ 28s 257ms/step - accuracy: 0.6391 - loss: 0.9242 - val_accuracy: 0.6314 - val_loss: 0.9355


In [29]:
CNN_model.save("mymodel.keras", save_format="keras_v3")

In [31]:
from tensorflow.keras.models import load_model

model = load_model("mymodel.keras")

In [32]:
from google.colab import files
files.download("mymodel.keras")   # أو mymodel.keras


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>